In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace, RDFS, OWL
from rdflib.namespace import XSD, RDFS

#### Step 1: Define ontology 

I chose to have only one class for all players, https://dbpedia.org/page/Football_player 

The case can be made that each position can be a different class. This is possible because dbpedia does indeed have different classes for each position. I chose not to do this because:

(1) Many statistics and properties are shared between both types of players (club, matches played, starts, etc.)

(2) For my use case, which is predicting which players will be at the same club at a given year, the type of player shouldn't make a difference to the predictions.  

(3) Based on the way I have cleaned the data, there can be no inconsistencies. That is, outfield players cannot have goalkeeper statistics and vice versa.  

Cons:

(1) Different players share the same super class. Which means there is a compromise in the semantics, since I chose to ignore subclasses that exist.

Note: queries can be made to find goalkeepers or other positions, it would just be querying the property BestPosition of each player, instead of type of node itself. If I was just building a KG I would have to be more stringent while defining my main, sub classes. 

In [2]:
# Define namespaces 

sofifa = Namespace("https://sofifa.com")
dbpediaclass = Namespace("https://dbpedia.org/page")
wikidata = Namespace("https://www.wikidata.org/wiki/")
sportsschema= Namespace("https://sportschema.org/ontologies/soccer")
dbpedia = Namespace("https://dbpedia.org/property")

g = Graph()
g.bind("sofifa", sofifa)
g.bind("dbpediaclass", dbpediaclass)
g.bind("wikidata", wikidata)
g.bind("sportsschema", sportsschema)
g.bind("dbpedia", dbpedia)

In [3]:
# Add main class 

player_uri = sofifa['player']
g.add((player_uri, RDF.type, dbpediaclass['Football_player']))

# Add player properties that don't change in 2 years (name, height, etc.)

constant_literals = {'Property:P1477':'name', 'Property:P27':'nationality', 'Property:P413':'position',
                     'Property:P8006':'preferredfoot'}

for key, value in constant_literals.items():
    g.add((URIRef(wikidata[key]), RDF.type, RDF.Property))
    g.add((URIRef(wikidata[key]), RDFS.domain, player_uri))
    g.add((URIRef(wikidata[key]), RDFS.range, RDFS.Literal))
    g.add((URIRef(wikidata[key]), RDFS.label, Literal(value)))

g.add((URIRef(wikidata['Property:P2048']), RDF.type, RDF.Property))
g.add((URIRef(wikidata['Property:P2048']), RDFS.domain, player_uri))
g.add((URIRef(wikidata['Property:P2048']), RDFS.range, XSD.float))
g.add((URIRef(wikidata['Property:P2048']), RDFS.label, Literal('height')))
      
g.add((URIRef(wikidata['Q21821348']), RDF.type, RDF.Property))
g.add((URIRef(wikidata['Q21821348']), RDFS.domain, player_uri))
g.add((URIRef(wikidata['Q21821348']), RDFS.range, XSD.dateTime))
g.add((URIRef(wikidata['Q21821348']), RDFS.label, Literal('birthyear')))


<Graph identifier=N29de5d0e2146418aa1789015bef1a7b1 (<class 'rdflib.graph.Graph'>)>

In [4]:
# Add fifa properties that change every year 

dbpedia_properties = [ "90s", "IntReputation", "Potential", "attackingworkrate", "contractuntil",
                       "defending", "defensiveworkrate", "dribbling", "gkdiving", "gkhandling",
                       "gkkicking", "gkpositioning", "gkreflexes", "matchesplayed", "minutes",
                       "overallRating", "pace", "passing", "physicality", "shooting", "skillmoves",
                       "starts", "value" ]

for prop in dbpedia_properties:
    g.add((URIRef(dbpedia[prop]), RDF.type, RDF.Property))
    g.add((URIRef(dbpedia[prop]), RDFS.domain, player_uri))
    g.add((URIRef(dbpedia[prop]), RDFS.range, RDF.Seq))

In [5]:
# Add statistics properties that change every year

sportsschema_properties = [ "aerialsWon", "assistsTotal", "clearancesSuccessful", "foulsCommited",
                            "foulsSuffered", "goalsTotal", "interceptions", "passesCompleteLong",
                            "passesCompletePercentage", "shotsOnGoalTotal", "touches" ]

for prop in sportsschema_properties:
    g.add((URIRef(sportsschema[prop]), RDF.type, RDF.Property))
    g.add((URIRef(sportsschema[prop]), RDFS.domain, player_uri))
    g.add((URIRef(sportsschema[prop]), RDFS.range, RDF.Seq))


g.add((URIRef(wikidata['Property:P54']), RDF.type, RDF.Property))
g.add((URIRef(wikidata['Property:P54']), RDFS.domain, player_uri))
g.add((URIRef(wikidata['Property:P54']), RDFS.range, RDF.Seq))
g.add((URIRef(wikidata['Property:P54']), RDFS.label, Literal('club')))

<Graph identifier=N29de5d0e2146418aa1789015bef1a7b1 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Define a local namespace & make all properties and classes equivalent to make querying easier
fb = Namespace("https://footballerontology.com")
g.bind("fb", fb)

# Link main player class
g.add((sofifa.player, OWL.equivalentClass, fb.Player))

# Link properties 
for prop in dbpedia_properties:
    g.add((URIRef(dbpedia[prop]), OWL.equivalentProperty, URIRef(fb[prop])))

for prop in sportsschema_properties:
    g.add((URIRef(sportsschema[prop]), OWL.equivalentProperty, URIRef(fb[prop])))

for key, value in constant_literals.items():
    g.add((URIRef(wikidata[key]), OWL.equivalentProperty, URIRef(fb[value])))

g.add((URIRef(wikidata['Property:P2048']), OWL.equivalentProperty, URIRef(fb['height'])))
g.add((URIRef(wikidata['Property:P54']), OWL.equivalentProperty, URIRef(fb['club'])))
g.add((URIRef(wikidata['Q21821348']), OWL.equivalentProperty, URIRef(fb['birthyear'])))

g.serialize(destination='../kg/schema.ttl', format='turtle')

<Graph identifier=N29de5d0e2146418aa1789015bef1a7b1 (<class 'rdflib.graph.Graph'>)>

#### Step 2: Populate KG 